In [0]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import roc_curve,auc
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
url='https://raw.githubusercontent.com/scarface961/Bank-Marketting/master/bank-full.csv'
data = pd.read_csv(url,delimiter=';')
#data1 = pd.read_csv("D:/Data Mining and Concepts Learning/Project/bank/bank.csv")

In [30]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [0]:
data=pd.get_dummies(data, columns=["job","marital","education","default","housing","loan","contact","month","poutcome"])

labelencoder = LabelEncoder()
data['y'] = labelencoder.fit_transform(data['y'])

In [32]:
data

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,17,977,3,-1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
45207,71,1729,17,456,2,-1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
45208,72,5715,17,1127,5,184,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
45209,57,668,17,508,4,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [0]:
def epoch (history):
  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [0]:
def dPCA(X_train,X_test):
    pca = PCA(n_components= 6)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    return X_train,X_test


In [0]:
def dSMOTE(X_train,y_train):
    smt = SMOTE()
    X_train, y_train = smt.fit_sample(X_train, y_train)
    return X_train, y_train

In [0]:
def percentage_split_NN(percentage_split,X,Y):
  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)

  model= create_NN_model()
  history = model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=150, batch_size=100)
  pred = model.predict_classes(x_test)
  tn,fp,fn,tp = confusion_matrix(y_test,pred).ravel()
  print("Percentage Split")
  Confusion_matrix(tn,fp,fn,tp)
  #epoch(history)
  

In [0]:
def crossvalidate_NN(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    model= create_NN_model()
    history = model.fit(X[train], Y[train],validation_data=(X[test],Y[test]),epochs=150, batch_size=100)
    pred = model.predict_classes(X[test])
    #evaluate the model
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],pred).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1

    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def percentage_split_NN_PCA(percentage_split,X,Y):
  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
  x_train,x_test = dPCA(x_train,x_test)
  model= create_NN_model()
  history = model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=150, batch_size=100)
  pred = model.predict_classes(x_test)
  tn,fp,fn,tp = confusion_matrix(y_test,pred).ravel()
  print("Percentage Split")
  Confusion_matrix(tn,fp,fn,tp)
  #epoch(history)
  

In [0]:
def crossvalidate_NN_PCA(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    x_train = X[train]
    x_test = X[test]
    x_train,x_test= dPCA(x_train,x_test)
    model= create_NN_model()
    history = model.fit(x_train, Y[train],validation_data=(x_test,Y[test]),epochs=150, batch_size=100)
    pred = model.predict_classes(x_test)
    #evaluate the model
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],pred).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1

    scores = model.evaluate(x_test, Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)

In [0]:
def percentage_split_NN_SMOTE(percentage_split,X,Y):
  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
  x_train,y_train = dSMOTE(x_train,y_train)
  model= create_NN_model()
  history = model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=150, batch_size=100)
  pred = model.predict_classes(x_test)
  fpr_nn,tpr_nn ,thresholds_nn  = roc_curve(y_test,pred)
  auc_nn = auc(fpr_nn, tpr_nn)
  tn,fp,fn,tp = confusion_matrix(y_test,pred).ravel()

  print("Percentage Split")
  Confusion_matrix(tn,fp,fn,tp)
  print("AUC: "+repr(auc_nn))
  return  fpr_nn,tpr_nn ,thresholds_nn,auc_nn
  #epoch(history)
  

In [0]:
def crossvalidate_NN_SMOTE(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  fpr_nn = 0
  tpr_nn = 0
  thresholds_nn = 0
  auc_nn=0
  for train, test in kfold.split(X, Y):
    # x_train = X[train]
    # x_test = X[test]
    x_train,y_train= dSMOTE(X[train],Y[train])
    model= create_NN_model()
    history = model.fit(x_train, y_train,validation_data=(X[test],Y[test]),epochs=150, batch_size=100)
    pred = model.predict_classes(X[test])
    #evaluate the model
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],pred).ravel()
    fpr_nn1,tpr_nn1 ,thresholds_nn1  = roc_curve(Y[test],pred)
    auc_nn1 = auc(fpr_nn1, tpr_nn1)
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1
    fpr_nn = fpr_nn + fpr_nn1
    tpr_nn =  tpr_nn + tpr_nn1
    thresholds_nn =thresholds_nn + thresholds_nn1 
    auc_nn=auc_nn + auc_nn1

    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  fpr_nn = fpr_nn/folds
  tpr_nn = tpr_nn/folds
  thresholds_nn = thresholds_nn/folds
  auc_nn = auc_nn/folds 
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)
  print("AUC: "+repr(auc_nn))
  return  fpr_nn,tpr_nn ,thresholds_nn,auc_nn

In [0]:
def percentage_split_SVM(percentage_split,X,Y):

  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
  clf = svm.SVC(kernel="linear")
  clf.fit(x_train, y_train)
  predict_svm = clf.predict(x_test)
  Confusion_matrix(y_test,predict_svm)


In [0]:
def crossvalidate_SVM(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    clf = svm.SVC(kernel="linear")
    clf.fit(X[train], Y[train])
    predict_svm = clf.predict(X[test])
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],predict_svm).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1


    print("Acc: "+repr(accuracy_score(Y[test],predict_svm)))

  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def percentage_split_SVM_SMOTE(percentage_split,X,Y):

    x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
    x_train,y_train = dSMOTE(x_train,y_train)
    clf = svm.SVC(kernel="rbf")
    clf.fit(x_train, y_train)
    predict_svm = clf.predict(x_test)
    tn,fp,fn,tp = confusion_matrix(y_test,predict_svm).ravel()
    fpr_svm,tpr_svm ,thresholds_svm  = roc_curve(y_test,predict_svm)
    auc_svm = auc(fpr_svm, tpr_svm)
    tn,fp,fn,tp = confusion_matrix(y_test,predict_svm).ravel()

    print("Percentage Split")
    Confusion_matrix(tn,fp,fn,tp)
    print("AUC: "+repr(auc_svm))
    return  fpr_svm,tpr_svm ,thresholds_svm,auc_svm

In [0]:
def crossvalidate_SVM_Smote(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  fpr_svm = 0
  tpr_svm = 0
  thresholds_svm = 0
  auc_svm=0
  for train, test in kfold.split(X, Y):
    x_train = X[train]
    y_train = Y[train]
    x_train,y_train= dSMOTE(x_train,y_train)
    clf = svm.SVC(kernel="rbf")
    clf.fit(x_train, y_train)
    predict_svm = clf.predict(X[test])
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],predict_svm).ravel()
    fpr_svm1,tpr_svm1 ,thresholds_svm1  = roc_curve(Y[test],predict_svm)
    auc_svm1 = auc(fpr_svm1, tpr_svm1)
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1
    fpr_svm = fpr_svm + fpr_svm1
    tpr_svm =  tpr_bayes + tpr_svm1
    thresholds_svm =thresholds_svm + thresholds_svm1 
    auc_svm=auc_svm + auc_svm1


    print("Acc: "+repr(accuracy_score(Y[test],predict_svm)))

  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  fpr_svm = fpr_svm/folds
  tpr_svm = tpr_svm/folds
  thresholds_svm = thresholds_svm/folds
  auc_svm = auc_svm/folds 
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)
  print("AUC: "+repr(auc_svm))
  return  fpr_svm,tpr_svm ,thresholds_svm,auc_svm

In [0]:
def Confusion_matrix(tn, fp, fn, tp):
  #tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
  print("Confusion Matrix")
  print("   yes     no   <<-- Classified as   ")
  # print("   %.2f    %.2f  yes" %(tp,fn))
  # print("   %.2f    %.2f  no" %(fp,tn))
  print("   "+repr(int(tp))+"    "+repr(int(fn))+"        yes")
  print("   "+repr(int(fp))+"    "+repr(int(tn))+"        no")
  accuracy = float((tp+tn)/(tp+tn+fp+fn))
  specificity = float(tn / (tn+fp))
  sensitivity = float(tp / (tp+fn))
  
  if (tp+fp)>=0:
    precision = float(tp / (tp+fp))
    f_score = (2*precision*sensitivity)/(precision+sensitivity) 
  else:
    precision = 0
    f_score = "?"
 
  #print(classification_report(y_test,predictions))
  print("Accuracy: "+repr(accuracy))
  print("Sensitivity: "+repr(sensitivity))
  print("Specificity: "+repr(specificity))
  print("Precision: "+repr(precision))
  print("f_score: "+repr(f_score))

In [0]:
def create_NN_model():
  #Keras Multi Layer
  model = Sequential()
  model.add(Dense(5,input_dim=51,activation='tanh'))
  model.add(Dense(5, activation='tanh'))
  model.add(Dense(1, activation='sigmoid'))
  #keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
  #sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
  a = optimizers.adam(lr=0.007)
  model.compile(loss='binary_crossentropy', optimizer=a, metrics=['accuracy'])
  return model


In [46]:
# Splitting the data labels and dataset
Y = data.y.values
X= data.drop('y',axis = 1).values
# X = data.iloc[:,0:53]
# Y = data.iloc[:,52:53]
# print(X)
# #Normalize
# from sklearn import preprocessing
# X = preprocessing.scale(X)
# print(Y)
from sklearn import preprocessing
X = preprocessing.scale(X)
X



array([[ 1.60696496,  0.25641925, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [ 0.28852927, -0.43789469, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.74738448, -0.44676247, -1.29847633, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       ...,
       [ 2.92540065,  1.42959305,  0.14341818, ..., -0.20597248,
         5.37784754, -2.11631591],
       [ 1.51279098, -0.22802402,  0.14341818, ..., -0.20597248,
        -0.185948  ,  0.47251925],
       [-0.37068857,  0.52836436,  0.14341818, ...,  4.85501757,
        -0.185948  , -2.11631591]])

In [0]:
percentage_split_NN(0.25,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Train on 33908 samples, validate on 11303 samples
Epoch 1/150
33908/33908 [==============================] - 1s 23us/step - loss: 0.2529 - acc: 0.8904 - val_loss: 0.2251 - val_acc: 0.8974
Epoch 2/150
33908/33908 [==============================] - 1s 15us/step - loss: 0.2165 - acc: 0.9039 - val_loss: 0.2163 - val_acc: 0.9010
Epoch 3/150
33908/33908 [==============================] - 1s 16us/step - loss: 0.2098 - acc: 0.9064 - val_loss: 0.2149 - val_acc: 0.9006
Epoch 4/150
33908/33908 [==============================] - 1s 16us/step - loss: 0.2053 - acc: 0.9080 - val_loss: 0.2147 - val_acc: 0.9013
Epoch 5/150
33908/33908 [==============================] - 1s 18us/step - loss: 0.2016 - acc: 0.9108 - val_loss: 0.2117 - val_acc: 0.9023
Epoch 6/150
33908/33908 [==============================] - 1s 17us/step - loss: 0.1986 - acc: 0.9117 - val_loss: 0.2140 - val_acc: 0.9023
Epoch 7/150
33908/33908 [==============================] - 1s 16us/step - loss: 0.1959 - acc: 0.9135 - val_loss: 0.2128 - 

In [0]:
crossvalidate_NN(10,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Train on 40689 samples, validate on 4522 samples
Epoch 1/150
40689/40689 [==============================] - 6s 141us/step - loss: 0.2573 - acc: 0.8895 - val_loss: 0.2255 - val_acc: 0.9027
Epoch 2/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2171 - acc: 0.9012 - val_loss: 0.2155 - val_acc: 0.9045
Epoch 3/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2118 - acc: 0.9027 - val_loss: 0.2112 - val_acc: 0.9034
Epoch 4/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2097 - acc: 0.9035 - val_loss: 0.2111 - val_acc: 0.9067
Epoch 5/150
40689/40689 [==============================] - 1s 22us/step - loss: 0.2078 - acc: 0.9039 - val_loss: 0.2120 - val_acc: 0.9054
Epoch 6/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2067 - acc: 0.9057 - val_loss: 0.2126 - val_acc: 0.9076
Epoch 7/150
40689/40689 [==============================] - 1s 21us/step - loss: 0.2065 - acc: 0.9049 - val_loss: 0.2096 - 

In [19]:
percentage_split_NN_PCA(0.25,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')




Train on 33908 samples, validate on 11303 samples
Epoch 1/150





33908/33908 [==============================] - 1s 23us/step - loss: 0.3686 - acc: 0.8585 - val_loss: 0.3165 - val_acc: 0.8859
Epoch 2/150
33908/33908 [==============================] - 1s 15us/step - loss: 0.3146 - acc: 0.8872 - val_loss: 0.3148 - val_acc: 0.8841
Epoch 3/150
33908/33908 [==============================] - 0s 14us/step - loss: 0.3115 - acc: 0.8865 - val_loss: 0.3121 - val_acc: 0.8846
Epoch 4/150
33908/33908 [==============================] - 1s 15us/step - loss: 0.3098 - acc: 0.8867 - val_loss: 0.3116 - val_acc: 0.8860
Epoch 5/150
33908/33908 [==============================] - 1s 15us/step - loss: 0.3091 - acc: 0.8858 - val_loss: 0.3111 - val_acc: 0.8827
Epoch 6/150
33908/33908 [==============================] - 1s 15us/step - loss: 0.3089 - acc: 0.8860 - val_loss: 0.3115 - val_acc: 0.8826
Epoch 7/150
33908/33908 [==============================] - 0s 15us/step - loss: 0.3083 - acc: 0.8870 - val_loss: 0

In [20]:
crossvalidate_NN_PCA(10,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Train on 40689 samples, validate on 4522 samples
Epoch 1/150
40689/40689 [==============================] - 1s 19us/step - loss: 0.3348 - acc: 0.8761 - val_loss: 0.3214 - val_acc: 0.8824
Epoch 2/150
40689/40689 [==============================] - 1s 13us/step - loss: 0.3127 - acc: 0.8854 - val_loss: 0.3186 - val_acc: 0.8837
Epoch 3/150
40689/40689 [==============================] - 1s 13us/step - loss: 0.3114 - acc: 0.8845 - val_loss: 0.3173 - val_acc: 0.8839
Epoch 4/150
40689/40689 [==============================] - 1s 13us/step - loss: 0.3112 - acc: 0.8846 - val_loss: 0.3178 - val_acc: 0.8819
Epoch 5/150
40689/40689 [==============================] - 1s 13us/step - loss: 0.3109 - acc: 0.8847 - val_loss: 0.3164 - val_acc: 0.8830
Epoch 6/150
40689/40689 [==============================] - 1s 15us/step - loss: 0.3103 - acc: 0.8850 - val_loss: 0.3149 - val_acc: 0.8861
Epoch 7/150
40689/40689 [==============================] - 1s 14us/step - loss: 0.3097 - acc: 0.8858 - val_loss: 0.3141 - v

In [47]:
fpr_nn_p,tpr_nn_p ,thresholds_nn_p,auc_nn_p=percentage_split_NN_SMOTE(0.25,X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 59882 samples, validate on 11303 samples
Epoch 1/150
59882/59882 [==============================] - 3s 46us/step - loss: 0.3598 - acc: 0.8466 - val_loss: 0.3613 - val_acc: 0.8263
Epoch 2/150
59882/59882 [==============================] - 1s 17us/step - loss: 0.3136 - acc: 0.8711 - val_loss: 0.3614 - val_acc: 0.8283
Epoch 3/150
59882/59882 [==============================] - 1s 18us/step - loss: 0.3037 - acc: 0.8764 - val_loss: 0.3599 - val_acc: 0.8284
Epoch 4/150
59882/59882 [==============================] - 1s 17us/step - loss: 0.2993 - acc: 0.8791 - val_loss: 0.3326 - val_acc: 0.8368
Epoch 5/150
59882/59882 [==============================] - 1s 17us/step - loss: 0.2968 - acc: 0.8798 - val_loss: 0.3394 - val_acc: 0.8331
Epoch 6/150
59882/59882 [==============================] - 1s 18us/step - loss: 0.2950 - acc: 0.8802 - val_loss: 0.3529 - val_acc: 0.8328
Epoch 7/150
59882/59882 [==============================] - 1s 18us/step - loss: 0.2945 - acc: 0.8813 - val_loss: 0.3461 - 

In [51]:
fpr_nn_c,tpr_nn_c ,thresholds_nn_c,auc_nn_c=crossvalidate_NN_SMOTE(10,X,Y)
print()
print()
print()
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 71858 samples, validate on 4522 samples
Epoch 1/150
71858/71858 [==============================] - 3s 44us/step - loss: 0.3632 - acc: 0.8452 - val_loss: 0.3776 - val_acc: 0.8253
Epoch 2/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.3182 - acc: 0.8712 - val_loss: 0.3572 - val_acc: 0.8339
Epoch 3/150
71858/71858 [==============================] - 1s 16us/step - loss: 0.3071 - acc: 0.8764 - val_loss: 0.3475 - val_acc: 0.8366
Epoch 4/150
71858/71858 [==============================] - 1s 18us/step - loss: 0.3037 - acc: 0.8770 - val_loss: 0.3410 - val_acc: 0.8470
Epoch 5/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.3015 - acc: 0.8784 - val_loss: 0.3434 - val_acc: 0.8430
Epoch 6/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.2996 - acc: 0.8786 - val_loss: 0.3591 - val_acc: 0.8317
Epoch 7/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.2989 - acc: 0.8791 - val_loss: 0.3369 - v

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 71858 samples, validate on 4521 samples
Epoch 1/150
71858/71858 [==============================] - 3s 45us/step - loss: 0.3522 - acc: 0.8518 - val_loss: 0.3745 - val_acc: 0.8319
Epoch 2/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.3129 - acc: 0.8732 - val_loss: 0.3712 - val_acc: 0.8246
Epoch 3/150
71858/71858 [==============================] - 1s 18us/step - loss: 0.3066 - acc: 0.8761 - val_loss: 0.3871 - val_acc: 0.8197
Epoch 4/150
71858/71858 [==============================] - 1s 17us/step - loss: 0.3039 - acc: 0.8786 - val_loss: 0.3584 - val_acc: 0.8303
Epoch 5/150
71858/71858 [==============================] - 1s 16us/step - loss: 0.3023 - acc: 0.8784 - val_loss: 0.3598 - val_acc: 0.8211
Epoch 6/150
71858/71858 [==============================] - 1s 16us/step - loss: 0.3007 - acc: 0.8792 - val_loss: 0.3536 - val_acc: 0.8326
Epoch 7/150
71858/71858 [==============================] - 1s 16us/step - loss: 0.3001 - acc: 0.8792 - val_loss: 0.3610 - v

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


...

Train on 71860 samples, validate on 4521 samples
Epoch 1/150
71860/71860 [==============================] - 4s 49us/step - loss: 0.3559 - acc: 0.8492 - val_loss: 0.3841 - val_acc: 0.8182
Epoch 2/150
71860/71860 [==============================] - 1s 17us/step - loss: 0.3197 - acc: 0.8682 - val_loss: 0.3359 - val_acc: 0.8447
Epoch 3/150
71860/71860 [==============================] - 1s 17us/step - loss: 0.3120 - acc: 0.8721 - val_loss: 0.3530 - val_acc: 0.8388
Epoch 4/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.3094 - acc: 0.8738 - val_loss: 0.3239 - val_acc: 0.8474
Epoch 5/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.3076 - acc: 0.8748 - val_loss: 0.3571 - val_acc: 0.8388
Epoch 6/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.3066 - acc: 0.8755 - val_loss: 0.3705 - val_acc: 0.8284
Epoch 7/150
71860/71860 [==============================] - 1s 19us/step - loss: 0.3052 - acc: 0.8778 - val_loss: 0.3119 - v

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 71860 samples, validate on 4521 samples
Epoch 1/150
71860/71860 [==============================] - 4s 52us/step - loss: 0.3562 - acc: 0.8466 - val_loss: 0.3660 - val_acc: 0.8264
Epoch 2/150
71860/71860 [==============================] - 1s 20us/step - loss: 0.3149 - acc: 0.8717 - val_loss: 0.3719 - val_acc: 0.8281
Epoch 3/150
71860/71860 [==============================] - 1s 17us/step - loss: 0.3050 - acc: 0.8756 - val_loss: 0.3312 - val_acc: 0.8405
Epoch 4/150
71860/71860 [==============================] - 1s 17us/step - loss: 0.3009 - acc: 0.8785 - val_loss: 0.3557 - val_acc: 0.8330
Epoch 5/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.2990 - acc: 0.8786 - val_loss: 0.3706 - val_acc: 0.8286
Epoch 6/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.2975 - acc: 0.8794 - val_loss: 0.3597 - val_acc: 0.8410
Epoch 7/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.2961 - acc: 0.8801 - val_loss: 0.3462 - v

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 71860 samples, validate on 4521 samples
Epoch 1/150
71860/71860 [==============================] - 4s 54us/step - loss: 0.3570 - acc: 0.8471 - val_loss: 0.3573 - val_acc: 0.8321
Epoch 2/150
71860/71860 [==============================] - 1s 20us/step - loss: 0.3173 - acc: 0.8685 - val_loss: 0.3593 - val_acc: 0.8381
Epoch 3/150
71860/71860 [==============================] - 1s 20us/step - loss: 0.3101 - acc: 0.8720 - val_loss: 0.3346 - val_acc: 0.8507
Epoch 4/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.3053 - acc: 0.8763 - val_loss: 0.3651 - val_acc: 0.8370
Epoch 5/150
71860/71860 [==============================] - 1s 18us/step - loss: 0.3015 - acc: 0.8777 - val_loss: 0.3259 - val_acc: 0.8507
Epoch 6/150
71860/71860 [==============================] - 1s 19us/step - loss: 0.2982 - acc: 0.8790 - val_loss: 0.3271 - val_acc: 0.8500
Epoch 7/150
71860/71860 [==============================] - 1s 19us/step - loss: 0.2968 - acc: 0.8787 - val_loss: 0.3171 - v

In [54]:
fpr_svm_p,tpr_svm_p ,thresholds_svm_p,auc_svm_p = percentage_split_SVM_SMOTE(0.25,X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Percentage Split
Confusion Matrix
   yes     no   <<-- Classified as   
   1054    268        yes
   1267    8714        no
Accuracy: 0.8641953463682208
Sensitivity: 0.7972768532526475
Specificity: 0.8730588117423104
Precision: 0.45411460577337354
f_score: 0.5786439747460884
AUC: 0.835167832497479


In [55]:
fpr_svm_c,tpr_svm_c ,thresholds_svm_c,auc_svm_c=crossvalidate_SVM_Smote(10,X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Acc: 0.8668730650154799


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Acc: 0.8615350586153506


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


...

Acc: 0.8701614687016147


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Acc: 0.8595443485954435


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Acc: 0.864852908648529
Cross-validation
Confusion Matrix
   yes     no   <<-- Classified as   
   418    110        yes
   503    3488        no
Accuracy: 0.8641702240605161
Sensitivity: 0.7905086027604463
Specificity: 0.873929161865638
Precision: 0.45376600824831775
f_score: 0.5765703647521202
AUC: 0.0


In [0]:
# clf = svm.SVC(kernel="linear")
# clf.fit(x_train, y_train)
# predict_svm = clf.predict(x_test)

In [0]:
# Confusion_matrix(y_test,predict_svm)

Confusion Matrix
   0     1   << Classified as   
   29    101
   35    966
Accuracy: 0.8797524314765695
Sensitivity: 0.2230769230769231
Specificity: 0.965034965034965


In [0]:
crossvalidate_SVM(10,X,Y)

Acc: 0.8741721854304636
Acc: 0.8761061946902655
Acc: 0.8849557522123894
Acc: 0.8938053097345132
Acc: 0.8915929203539823
Acc: 0.8738938053097345
Acc: 0.8761061946902655
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Cross-validation
Confusion Matrix
   0     1   << Classified as   
   15    37
   14    385
Accuracy: 0.8815207780725022
Sensitivity: 0.28982725527831094
Specificity: 0.9627500000000001
